## Step 1: Import Packages and Declare Constants

In [1]:
import boto3
import sagemaker
import datetime as dt
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
#Replace this value with the S3 Bucket Created
default_bucket = "sagemaker-us-east-1-863828306212"

In [3]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
sklearn_processor_version="0.23-1"
model_package_group_name="ChurnModelPackageGroup"
pipeline_name= "ChurnModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

## Step 2: Generate Baseline Dataset

Baseline Data will be used as part of SageMaker Clarify Step to generate SHAP Values 

In [4]:
# Preprocess the dataset
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    # Convert to datetime columns
    df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
    df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
    # Drop Rows with null values
    df = df.dropna()    
    # Create Column which gives the days between the last order and the first order
    df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
    # Create Column which gives the days between when the customer record was created and the first order
    df['created'] = pd.to_datetime(df['created'])
    df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
    # Drop Columns
    df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
    ## Apply one hot encoding on favday and city columns
    df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
    categorical_cols = ['favday_Friday', 'favday_Monday', 'favday_Saturday', 'favday_Sunday',
                        'favday_Thursday', 'favday_Tuesday', 'favday_Wednesday', 'city_BLR',
                        'city_BOM', 'city_DEL', 'city_MAA']
    df[categorical_cols] = df[categorical_cols].replace({True: 1, False: 0})
    return df

In [5]:
baseline_data = preprocess_data("data/storedata_total.csv")
baseline_data.pop("retained")
baseline_sample = baseline_data.sample(frac=0.0002)

/tmp/ipykernel_4864/3676691913.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
/tmp/ipykernel_4864/3676691913.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
/tmp/ipykernel_4864/3676691913.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created'] = pd.to_datetime(df['created'])


In [6]:
pd.DataFrame(baseline_sample).to_csv("data/baseline.csv",header=False,index=False)

## Step 3: Generate Batch Dataset

In [7]:
batch_data = preprocess_data("data/storedata_total.csv")
batch_data.pop("retained")
batch_sample = batch_data.sample(frac=0.2)

/tmp/ipykernel_4864/3676691913.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
/tmp/ipykernel_4864/3676691913.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
/tmp/ipykernel_4864/3676691913.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created'] = pd.to_datetime(df['created'])


In [8]:
pd.DataFrame(batch_sample).to_csv("data/batch.csv", sep=',', header=False, index=False)

## Step 4: Copy Data and Scripts to S3 Bucket

In [9]:
s3_client = boto3.resource('s3')
s3_client.Bucket(default_bucket).upload_file("data/storedata_total.csv","data/storedata_total.csv")
s3_client.Bucket(default_bucket).upload_file("data/batch.csv","data/batch/batch.csv")
s3_client.Bucket(default_bucket).upload_file("data/baseline.csv","input/baseline/baseline.csv")

In [10]:
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/preprocess.py","input/code/preprocess.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/evaluate.py","input/code/evaluate.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/generate_config.py","input/code/generate_config.py")

## Step 5: Get the Pipeline Instance

In [11]:
from pipelines.customerchurn.pipeline import get_pipeline

pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


Default Bucket: sagemaker-us-east-1-863828306212
Model Package Group Name: ChurnModelPackageGroup
Pipeline Name: ChurnModelSMPipeline
Input Data: s3://sagemaker-us-east-1-863828306212/data/storedata_total.csv
Batch Data: s3://sagemaker-us-east-1-863828306212/data/batch/batch.csv


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [12]:
pipeline.definition()

'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "InputData", "Type": "String", "DefaultValue": "s3://sagemaker-us-east-1-863828306212/data/storedata_total.csv"}, {"Name": "BatchData", "Type": "String", "DefaultValue": "s3://sagemaker-us-east-1-863828306212/data/batch/batch.csv"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "ChurnModelProcess", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": {"Get": "Parameters.ProcessingInstanceType"}, "InstanceCount": {"Get": "Parameters.ProcessingInstanceCount"}, "VolumeSizeInGB": 30}}, "AppSpecification": {"ImageUri": "683313688378.d

## Step 5: Submit the pipeline to SageMaker and start execution

In [13]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:863828306212:pipeline/ChurnModelSMPipeline',
 'ResponseMetadata': {'RequestId': '4119cf1c-a38b-4c99-843d-c575bdfabaf2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4119cf1c-a38b-4c99-843d-c575bdfabaf2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Fri, 30 Aug 2024 19:17:10 GMT'},
  'RetryAttempts': 0}}

## Start Pipeline Execution

In [14]:
execution = pipeline.start()

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [15]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:863828306212:pipeline/ChurnModelSMPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:863828306212:pipeline/ChurnModelSMPipeline/execution/23gp42s2t9fc',
 'PipelineExecutionDisplayName': 'execution-1725045431678',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2024, 8, 30, 19, 17, 11, 596000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 8, 30, 19, 17, 11, 596000, tzinfo=tzlocal()),
 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::863828306212:assumed-role/AmazonSageMaker-ExecutionRole-20240828T140437/SageMaker',
   'PrincipalId': 'AROA4SIBN5ESFGUJ2NPH4:SageMaker'}},
 'LastModifiedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::863828306212:assumed-role/AmazonSageMaker-ExecutionRole-20240828T140437/SageMaker',
   'PrincipalId': 'AROA4SIBN5ESFGUJ2NPH4:SageMaker'}},
 'ResponseMetadata': {'RequestId': 'c6216e7f-5d0f-4d8c-9ada-02c71fa17eb0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-a

We can list the execution steps to check out the status and artifacts:

In [35]:
execution.list_steps()

[{'StepName': 'ClarifyProcessingStep',
  'StartTime': datetime.datetime(2024, 8, 30, 19, 27, 28, 477000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 8, 30, 19, 45, 3, 822000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:863828306212:processing-job/pipelines-23gp42s2t9fc-ClarifyProcessingSte-YWJwaKMNXO'}},
  'AttemptCount': 1},
 {'StepName': 'ChurnModelConfigFile',
  'StartTime': datetime.datetime(2024, 8, 30, 19, 24, 53, 751000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 8, 30, 19, 27, 27, 751000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:863828306212:processing-job/pipelines-23gp42s2t9fc-ChurnModelConfigFile-wR5xiHvMdF'}},
  'AttemptCount': 1},
 {'StepName': 'ChurnTransform',
  'StartTime': datetime.datetime(2024, 8, 30, 19, 24, 53, 751000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 8, 30, 19, 30, 25, 9